#**Import Libraries**

In [ ]:
pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.6 MB/s eta 0:00:00


In [ ]:
pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 25.3 MB/s eta 0:00:00


In [ ]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
#pip uninstall -y numpy gensim

In [ ]:
pip install numpy==1.24.0 gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.24.0 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.0 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4,

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
from underthesea import word_tokenize, pos_tag
from pyvi import ViTokenizer, ViPosTagger
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# thư viện NLP tiếng Việt
from tqdm import tqdm
import pickle
import unidecode
import gensim

# **Import Data**

In [19]:
#url = 'https://raw.githubusercontent.com/anhlam-road/Med_Specialty_Classifier/refs/heads/main/disease_11k.csv'
#url = 'https://raw.githubusercontent.com/anhlam-road/Med_Specialty_Classifier/refs/heads/main/train_d.csv'
url = 'https://raw.githubusercontent.com/anhlam-road/Med_Specialty_Classifier/refs/heads/main/raw_disease_updated.csv'
df = pd.read_csv(url)
df

,Disease,Department,Symptom,Category
0,Ảo giác,Tâm thần,"Người bệnh có thể bị căng thẳng, mất ngủ, đờ đ...",Knownledge
1,Rối loạn đa nhân cách,Tâm thần,Người mắc bệnh rối loạn đa nhân cách thường có...,Knownledge
2,Rối loạn giấc ngủ,Tâm thần,Triệu chứng chung của những người bị mắc chứng...,Knownledge
3,Trầm cảm,Tâm thần,Bệnh nhân khi bị trầm cảm sẽ có các biểu hiện ...,Knownledge
4,Mất ngủ ở người cao tuổi,Tâm thần,Triệu chứng của mất ngủ ở người cao tuổi là kh...,Knownledge
...,...,...,...,...
28022,NaN,Dị ứng,"Tôi hay bị mẩn ngứa khi ăn cua, ghẹ (đặc biệt ...",News
28023,NaN,Hô hấp,Dấu hiệu của bệnh viêm phế quản là gì? .Viêm p...,News
28024,NaN,Nam khoa,Vợ chồng cháu đi khám hiếm muộn thì cần phải l...,News
28025,NaN,Nội tiết,Bệnh nhân tiền sử tiểu đường tuýp 2 nóng rát t...,News


In [20]:
# Thông tin về số dòng, số cột, kiểu dữ liệu
print("Thông tin tổng quan về DataFrame:")
print(df.info())

# Kiểm tra missing values
print("\nSố lượng giá trị bị thiếu trong mỗi cột:")
print(df.isnull().sum())

# Kiểm tra số lượng dòng trùng
duplicate_rows = df.duplicated().sum()
print(f"\nSố lượng dòng trùng lặp: {duplicate_rows}")

Thông tin tổng quan về DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28027 entries, 0 to 28026
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Disease     5029 non-null   object
 1   Department  28027 non-null  object
 2   Symptom     28027 non-null  object
 3   Category    28027 non-null  object
dtypes: object(4)
memory usage: 876.0+ KB
None

Số lượng giá trị bị thiếu trong mỗi cột:
Disease       22998
Department        0
Symptom           0
Category          0
dtype: int64

Số lượng dòng trùng lặp: 762


#**DATASET HOÀN CHỈNH**

In [ ]:
#CÁC HÀM XỬ LÝ NỘI DUNG SYMPTOM

In [ ]:
import re
from difflib import SequenceMatcher

# Hàm chuẩn hóa
def normalize_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text.strip()

# Hàm loại bỏ câu lặp
def remove_duplicate_sentences(text, similarity_threshold=0.85):
    if pd.isnull(text) or not isinstance(text, str):
        return ""

    sentences = re.split(r'(?<=[.!?])\s+', text)
    unique_sentences = []
    seen = []

    for sent in sentences:
        norm_sent = normalize_text(sent)
        is_duplicate = any(SequenceMatcher(None, norm_sent, s).ratio() > similarity_threshold for s in seen)

        if not is_duplicate:
            unique_sentences.append(sent.strip())
            seen.append(norm_sent)

    return ' '.join(unique_sentences)

# Hàm xử lý toàn diện
def clean_symptom_pipeline(text):
    if pd.isnull(text):
        return ""

    text = re.sub(r'([.?!])(?=\w)', r'\1 ', text)
    text = re.sub(r'([.?!])(?=\S)', r'\1 ', text)
    text = re.sub(r'\.{2,}', '.', text)
    text = re.sub(r'!{2,}', '!', text)
    text = re.sub(r'\?{2,}', '?', text)
    text = re.sub(r'\s+', ' ', text).strip()

    text = remove_duplicate_sentences(text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# Ví dụ áp dụng trên DataFrame
df1 = df.copy()
df1['Symptom_Cleaned'] = df1['Symptom'].apply(clean_symptom_pipeline)

# Xuất thử vài dòng
print(df1[['Disease', 'Symptom_Cleaned']].head())


                    Disease                                    Symptom_Cleaned
0                   Ảo giác  Người bệnh có thể bị căng thẳng, mất ngủ, đờ đ...
1     Rối loạn đa nhân cách  Người mắc bệnh rối loạn đa nhân cách thường có...
2         Rối loạn giấc ngủ  Triệu chứng chung của những người bị mắc chứng...
3                  Trầm cảm  Bệnh nhân khi bị trầm cảm sẽ có các biểu hiện ...
4  Mất ngủ ở người cao tuổi  Triệu chứng của mất ngủ ở người cao tuổi là kh...


In [ ]:
df1.to_csv('cleaned1_symptom_data.csv', index=False)

#**Pre-Processing**

In [21]:
# Xóa dòng chứa giá trị bị thiếu
#df1 = df.dropna()

df1 = df.drop('Disease', axis=1)

# Xóa dòng trùng lặp
df1 = df1.drop_duplicates()

# Kiểm tra missing values
print("\nSố lượng giá trị bị thiếu trong mỗi cột:")
print(df1.isnull().sum())

# Kiểm tra số lượng dòng trùng
duplicate_rows = df1.duplicated().sum()
print(f"\nSố lượng dòng trùng lặp: {duplicate_rows}")

print(df1.info())


Số lượng giá trị bị thiếu trong mỗi cột:
Department    0
Symptom       0
Category      0
dtype: int64

Số lượng dòng trùng lặp: 0
<class 'pandas.core.frame.DataFrame'>
Index: 27257 entries, 0 to 28026
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Department  27257 non-null  object
 1   Symptom     27257 non-null  object
 2   Category    27257 non-null  object
dtypes: object(3)
memory usage: 851.8+ KB
None


XỬ LÝ SYMPTOM

In [22]:
# Hàm xử lý dữ liệu trong cột Symptom
def preprocess_text(text):
    text = gensim.utils.simple_preprocess(text)  # Tiền xử lý văn bản
    text = ' '.join(text)  # Chuyển danh sách từ thành chuỗi
    text = ViTokenizer.tokenize(text)  # Tách từ tiếng Việt
    return text

In [28]:
df2 = df1.copy()
# Áp dụng xử lý lên cột Symptom
df2["Symptom"] = df2["Symptom"].apply(preprocess_text)

# Xem kết quả sau khi xử lý
df2.head()

,Department,Symptom,Category
0,Tâm thần,người_bệnh có_thể bị căng_thẳng mất_ngủ đờ_đẫn...,Knownledge
1,Tâm thần,người mắc bệnh rối_loạn đa nhân_cách thường có...,Knownledge
2,Tâm thần,triệu_chứng chung của những người bị mắc chứng...,Knownledge
3,Tâm thần,bệnh_nhân khi bị trầm_cảm sẽ có các biểu_hiện ...,Knownledge
4,Tâm thần,triệu_chứng của mất_ngủ người cao_tuổi là khó ...,Knownledge


XỬ LÝ DEPARTMENT

In [51]:
df3 = df2.copy()
print(df3["Department"].value_counts())

Department
Xét nghiệm              2740
Nội tiết                2033
Truyền nhiễm            1871
Tai mũi họng            1668
Dinh dưỡng              1499
Răng hàm mặt            1485
Da liễu                 1384
Hô hấp                  1373
Tiêu hóa                1186
Thần kinh               1101
Máu                     1007
Dị ứng                   992
Nội thần kinh            987
Mắt                      897
Tiết niệu                809
Tim mạch                 703
Ung thư                  644
Ngoại thần kinh          631
Tâm lý                   609
Tâm thần                 515
Mạch máu                 383
Ung bướu                 359
Cơ xương khớp            347
Tiêu hóa - Gan mật       347
Nam khoa                 302
Ngoại lồng ngực          277
Thận tiết niệu           135
Sức khỏe sinh sản        133
Da tóc móng              122
Di truyền                114
Nội tiết chuyển hóa       93
Sản phụ khoa              92
Sức khỏe giới tính        74
Nam học                   35
Nuô

In [52]:
# Tạo từ điển mapping cho các nhãn tương đồng và chuẩn hóa về chữ thường không dấu
department_mapping = {
    "Tim mạch": ["tim mach"],
    "Tiêu hoá": ["tieu hoa", "tieu hoa - gan mat"],
    "Cơ - Xương - Khớp": ["co xuong khop"],
    "Thận - Tiết niệu": ["tiet nieu", "than tiet nieu"],
    "Nội tiết": ["noi tiet", "noi tiet chuyen hoa"],
    "Di ứng": ["di ung"],
    "Truyền nhiễm": ["truyen nhiem"],
    "Da liễu": ["da lieu", "da toc mong"],
    "Thần kinh": ["noi than kinh", "ngoai than kinh", "than kinh"],
    "Tâm thần": ["tam than", "tam ly"],
    "Nhi": ["nhi", "nuoi day con"],
    "Ngoại lồng ngực": ["ngoai long nguc"],
    "Ung bướu": ["ung thu", "ung buou"],
    "Sức khoẻ giới tính": ["suc khoe gioi tinh", "suc khoe nam gioi", "suc khoe nu gioi", "suc khoe tinh duc", "nam hoc", "nam khoa"],
    "Phụ Sản": ["suc khoe sinh san", "san phu khoa", "vu nhu", "mang thai", "ho tro sinh san ivf"],
    "Tai - Mũi - Họng": ["tai mui hong"],
    "Răng - Hàm - Mặt": ["rang ham mat"],
    "Mắt": ["mat", "nhan khoa"],
    "Máu": ["mau", "mach mau", "mau mien dich"],
    "Xét nghiệm": ["xet nghiem"],
    "Chẩn đoán hình ảnh": ["chan doan hinh anh"],
    "Dinh dưỡng": ["dinh duong"],
    "Hô hấp": ["ho hap"],
}

# Hàm chuyển đổi chuỗi thành không dấu
def remove_accents(input_str):
    return unidecode.unidecode(input_str)

# Hàm chuẩn hóa theo từ điển mapping (không dấu)
def map_department(department):
    department = department.strip().lower()  # Chuyển về chữ thường và loại bỏ khoảng trắng thừa
    department = remove_accents(department)  # Loại bỏ dấu
    for key, values in department_mapping.items():
        # Kiểm tra nếu department có trong danh sách các giá trị đã chuẩn hóa không dấu
        if department in [remove_accents(val).lower() for val in values]:
            return key
    return department  # Nếu không có trong mapping thì giữ nguyên

# Áp dụng mapping vào cột Department
df3["Department"] = df3["Department"].apply(map_department)

# Kiểm tra lại các giá trị duy nhất sau khi mapping
print("Các giá trị duy nhất sau khi mapping:", df3["Department"].unique())

# Xem kết quả
df3.head()


Các giá trị duy nhất sau khi mapping: ['Tâm thần' 'Thận - Tiết niệu' 'Hô hấp' 'Thần kinh' 'Truyền nhiễm'
 'Ngoại lồng ngực' 'Sức khoẻ giới tính' 'Nội tiết' 'Tiêu hoá' 'Máu'
 'Tim mạch' 'Ung bướu' 'di truyen' 'Răng - Hàm - Mặt' 'Tai - Mũi - Họng'
 'Di ứng' 'Xét nghiệm' 'Mắt' 'Da liễu' 'Dinh dưỡng' 'Phụ Sản'
 'suc khoe tong quat' 'Chẩn đoán hình ảnh' 'dau mat co'
 'Cơ - Xương - Khớp' 'Nhi' 'bi quyet song khoe' 'giac ngu ngon'
 'dinh duong tiet che']


,Department,Symptom,Category,Department_Encoded
0,Tâm thần,người_bệnh có_thể bị căng_thẳng mất_ngủ đờ_đẫn...,Knownledge,44
1,Tâm thần,người mắc bệnh rối_loạn đa nhân_cách thường có...,Knownledge,44
2,Tâm thần,triệu_chứng chung của những người bị mắc chứng...,Knownledge,44
3,Tâm thần,bệnh_nhân khi bị trầm_cảm sẽ có các biểu_hiện ...,Knownledge,44
4,Tâm thần,triệu_chứng của mất_ngủ người cao_tuổi là khó ...,Knownledge,44


In [53]:
print(df3["Department"].value_counts())

Department
Xét nghiệm             2740
Thần kinh              2719
Nội tiết               2126
Truyền nhiễm           1871
Tai - Mũi - Họng       1668
Tiêu hoá               1533
Da liễu                1506
Dinh dưỡng             1499
Răng - Hàm - Mặt       1485
Máu                    1421
Hô hấp                 1373
Tâm thần               1124
Ung bướu               1003
Di ứng                  992
Thận - Tiết niệu        944
Mắt                     917
Tim mạch                703
Sức khoẻ giới tính      468
Cơ - Xương - Khớp       347
Phụ Sản                 288
Ngoại lồng ngực         277
di truyen               114
Nhi                      60
suc khoe tong quat       31
Chẩn đoán hình ảnh       28
dau mat co               10
bi quyet song khoe        7
giac ngu ngon             2
dinh duong tiet che       1
Name: count, dtype: int64


In [54]:
# Sklearn: Các mô-đun hỗ trợ Machine Learning
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.model_selection import train_test_split

LABEL ENCONDING

In [55]:
encoder = preprocessing.LabelEncoder()
df3["Department_Encoded"] = encoder.fit_transform(df3["Department"])

encoder.classes_

array(['Chẩn đoán hình ảnh', 'Cơ - Xương - Khớp', 'Da liễu', 'Di ứng',
       'Dinh dưỡng', 'Hô hấp', 'Máu', 'Mắt', 'Ngoại lồng ngực', 'Nhi',
       'Nội tiết', 'Phụ Sản', 'Răng - Hàm - Mặt', 'Sức khoẻ giới tính',
       'Tai - Mũi - Họng', 'Thần kinh', 'Thận - Tiết niệu', 'Tim mạch',
       'Tiêu hoá', 'Truyền nhiễm', 'Tâm thần', 'Ung bướu', 'Xét nghiệm',
       'bi quyet song khoe', 'dau mat co', 'di truyen',
       'dinh duong tiet che', 'giac ngu ngon', 'suc khoe tong quat'],
      dtype=object)

# SPLIT TRAIN - TEST

In [ ]:
# Đếm số dòng theo cột 'Category'
category_counts = df2['Category'].value_counts()

# In kết quả
print(category_counts)

Category
QA            6496
Knownledge    4683
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Chia dữ liệu theo Category
df_knownledge = df2[df2['Category'] == 'Knownledge']  # Toàn bộ Knownledge vào train
df_qa = df2[df2['Category'] == 'QA']  # Chia QA

# Chia 50% QA vào train, 50% vào test
df_qa_train, df_qa_test = train_test_split(df_qa, test_size=0.5, random_state=42)

# Gộp train set
df_train = pd.concat([df_knownledge, df_qa_train])
df_test = df_qa_test

# Lưu ra file CSV
df_train.to_csv("data_train.csv", index=False)
df_test.to_csv("data_test.csv", index=False)

print(f"Tập train: {df_train.shape[0]} dòng")
print(f"Tập test: {df_test.shape[0]} dòng")


Tập train: 7931 dòng
Tập test: 3248 dòng


In [ ]:
df_train.head()

,Department,Symptom,Category
6496,Ung thư,triệu_chứng ung_thư phổi những dấu_hiệu ung_th...,Knownledge
6497,Ung thư,triệu_chứng ung_thư gan nguyên_phát những dấu_...,Knownledge
6498,Ung thư,triệu_chứng ung_thư triệu_chứng của bệnh ung_t...,Knownledge
6499,Ung thư,triệu_chứng ung_thư dày những dấu_hiệu_triệu_c...,Knownledge
6500,Ung thư,triệu_chứng ung_thư tử_cung những triệu_chứng ...,Knownledge


In [ ]:
X_data = df_train.drop(["Department", "Category"], axis = 1)
y_data = df_train['Department']

In [ ]:
X_data.head()

,Symptom
6496,triệu_chứng ung_thư phổi những dấu_hiệu ung_th...
6497,triệu_chứng ung_thư gan nguyên_phát những dấu_...
6498,triệu_chứng ung_thư triệu_chứng của bệnh ung_t...
6499,triệu_chứng ung_thư dày những dấu_hiệu_triệu_c...
6500,triệu_chứng ung_thư tử_cung những triệu_chứng ...


In [ ]:
y_data.head()

,Department
6496,Ung thư
6497,Ung thư
6498,Ung thư
6499,Ung thư
6500,Ung thư


# TF - IDF VECTORS: TRANSFORM BY SVD TO DECREASE NUMBER OF DIMENSIONS
*   WORD LEVEL
*   NGRAM LEVEL
*   NGRAM CHAR LEVEL

In [ ]:
from underthesea import word_tokenize
from collections import Counter

# Gộp toàn bộ văn bản thành 1 chuỗi lớn
all_text = " ".join(X_data['Symptom'])

# Tách từ và tạo tập hợp từ duy nhất
unique_words = set(all_text.split())

# Kết quả
print(f"Tổng số từ duy nhất trong X_data đã qua xử lý: {len(unique_words)}")
print(f"Danh sách một số từ duy nhất: {list(unique_words)[:10]}")  # Hiển thị 10 từ đầu tiên


Tổng số từ duy nhất trong X_data đã qua xử lý: 13137
Danh sách một số từ duy nhất: ['tưởng_hóa', 'oscillopsia', 'bari', 'hoàn_toàn_thể_tích', 'trầm_cảm_phần', 'xây_dựng', 'nghiên_cứu_dụng', 'kim', 'lây', 'fournier']
